In [1]:
from src.rag.components.shared.io import IOManager

In [2]:
from pathlib import Path

In [3]:
output_path = Path.cwd().joinpath("datasets", "parsed_documents_with_embeddings")
io_manager = IOManager(output_path)

In [4]:
io_manager.document_path.exists()

True

In [5]:
chunk_size = 5
all_documents = []
for i in range(0, 20, chunk_size):
	documents = io_manager.load_documents(i, i + chunk_size)
	all_documents.extend(documents)

In [6]:
# generate a random number between 0 and 20
import random
random_number = random.randint(0, 20)

In [7]:
sample_doc = all_documents[random_number]

In [8]:
nodes = sample_doc.convert_to_milvus()

In [9]:
nodes[0].get("text")

'Model Complexity \nImpact On Bias And Variance <br><br>Increasing model complexity reduces bias by allowing the model to \ncapture more intricate patterns in the data, but it also increases \nvariance, making the model more sensitive to noise and potentially \nleading to overfitting. '

In [10]:
from src.rag.components.shared.databases.milvus import MilvusDatabase

In [19]:
COLLECTION_NAME = "my_collection"

In [24]:
milvus_client = MilvusDatabase(
    host="http://localhost:19530",
    vector_dimension=1024,
    collection_name=COLLECTION_NAME,
    token=""
)

In [25]:
milvus_client.connect()

2025-05-12 23:57:23 | INFO     | milvus_database:connect:18 | Connecting to Milvus...
2025-05-12 23:57:23 | INFO     | milvus_database:connect:22 | Connected to Milvus successfully.


In [26]:
milvus_client.create_index_if_not_exists()

2025-05-12 23:57:23 | INFO     | milvus_database:create_index_if_not_exists:34 | Collection 'my_collection' already exists.


In [42]:
milvus_client.client.query(collection_name=COLLECTION_NAME, output_fields=["count(*)"])

data: ["{'count(*)': 23050}"]

In [31]:
from sentence_transformers import SentenceTransformer

/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:

embedding_model_name = "intfloat/multilingual-e5-large"

In [33]:
embedding_model = SentenceTransformer(embedding_model_name)

In [47]:
test_questions = ["what is dependency ingestion?"]

In [48]:
question_embeddings  = embedding_model.encode(test_questions, convert_to_tensor=True)

In [49]:
question_embeddings = question_embeddings.cpu().numpy()



In [50]:
milvus_client.connect()

2025-05-13 00:04:48 | INFO     | milvus_database:connect:18 | Connecting to Milvus...
2025-05-13 00:04:48 | INFO     | milvus_database:connect:22 | Connected to Milvus successfully.


In [51]:
responses = milvus_client.client.search(
    collection_name=COLLECTION_NAME,
    data=question_embeddings,
    limit=5,
    output_fields=["text", "metadata"],
    params={"metric_type": "L2"}
)

In [52]:
len(responses)

1

In [53]:
responses

data: [[{'id': '3add1113-b283-4bc6-bd02-e54855beff6f', 'distance': 0.8309718370437622, 'entity': {'metadata': {'filename': 'Clean Architecture A Craftsman’s Guide to Software Structure and Design (Robert C. Martin) (Z-Library).pdf', 'num_pages': 429, 'coordinate_system': 'bottom-left', 'last_modified_date': '2024-12-18T00:00:00', 'last_accessed_date': '2025-02-09T00:00:00', 'creation_date': '2025-01-09T00:00:00', 'file_size': 6677970}, 'text': '11**DIP: The**<br><br>**D ependency**\n**I nversion Principle**<br><br><br><br>The Dependency Inversion Principle (DIP) tells us that the most flexible \nsystems are those in which source code dependencies refer only to \nabstractions, not to concretions. <br><br>In a statically typed language, like Java, this means that the use, import, \nand include statements should refer only to source modules containing \ninterfaces, abstract classes, or some other kind of abstract declaration. \nNothing concrete should be depended on. <br><br>**87**'}}, {'

In [54]:
for question, response in zip(test_questions, responses):
    print(f"Question: {question}")
    print("===" * 20)
    for result in response:
        print(result.id, result.distance, result.entity.get("text"))
        print(result.entity.get("metadata"))
        print("\n")
    print("===" * 20)

Question: what is dependency ingestion?
3add1113-b283-4bc6-bd02-e54855beff6f 0.8309718370437622 11**DIP: The**<br><br>**D ependency**
**I nversion Principle**<br><br><br><br>The Dependency Inversion Principle (DIP) tells us that the most flexible 
systems are those in which source code dependencies refer only to 
abstractions, not to concretions. <br><br>In a statically typed language, like Java, this means that the use, import, 
and include statements should refer only to source modules containing 
interfaces, abstract classes, or some other kind of abstract declaration. 
Nothing concrete should be depended on. <br><br>**87**
{'filename': 'Clean Architecture A Craftsman’s Guide to Software Structure and Design (Robert C. Martin) (Z-Library).pdf', 'num_pages': 429, 'coordinate_system': 'bottom-left', 'last_modified_date': '2024-12-18T00:00:00', 'last_accessed_date': '2025-02-09T00:00:00', 'creation_date': '2025-01-09T00:00:00', 'file_size': 6677970}


56bb8b56-dc53-4569-9078-925acada78

In [ ]:
### Need to come and chech the results and implement vector search.